In [1]:
import time
import requests
import pandas as pd

In [36]:
my_df = pd.read_csv('../Extract_citations/out', sep = '\t', dtype=str)
display(my_df)

,Citing DOI,Citation ID,CC paragraph,CC prev_next,IMRD,Section title,Cited title,DOI
0,10.3892/ol.2019.10700,b1-ol-0-0-10700,Osteosarcoma is the most common primary malign...,Osteosarcoma is the most common primary malign...,Introduction,Introduction,Osteosarcoma,10.1016/S0031-3955(05)70540-X
1,10.3892/ol.2019.10700,b2-ol-0-0-10700,Osteosarcoma is the most common primary malign...,Osteosarcoma is the most common primary malign...,Introduction,Introduction,Osteosarcoma incidence and survival rates from...,10.1002/cncr.24121
2,10.3892/ol.2019.10700,b3-ol-0-0-10700,Osteosarcoma is the most common primary malign...,Osteosarcoma is the most common primary malign...,Introduction,Introduction,Osteosarcoma (osteogenic sarcoma),10.1186/1750-1172-2-6
3,10.3892/ol.2019.10700,b4-ol-0-0-10700,Osteosarcoma is the most common primary malign...,"To date, the most common treatment for osteosa...",Introduction,Introduction,Current and future therapeutic approaches for ...,10.1080/14737140.2018.1413939
4,10.3892/ol.2019.10700,b5-ol-0-0-10700,Previous studies have revealed that the amount...,Previous studies have revealed that the amount...,Introduction,Introduction,The relationship between non-protein-coding DN...,10.1002/bies.20544
...,...,...,...,...,...,...,...,...
96244,10.1007/s12094-023-03189-3,CR240,"BC is the most common cancer among women, and ...",The often encountered drug resistance within B...,Conclusions,Conclusions,Current role of non-coding RNAs in the clinica...,10.1016/j.ncrna.2019.09.001
96245,10.1007/s12094-023-03189-3,CR241,"Herein, we summarized the dysregulated ncRNAs ...",This offers a novel approach of inhibition wit...,Conclusions,Conclusions,Preclinical pharmacokinetic studies of 3-deaza...,10.1016/j.ejps.2015.06.021
96246,10.1016/j.envres.2020.110265,bib2,Traces of SARS-CoV-2 coronavirus (its nucleic ...,Traces of SARS-CoV-2 coronavirus (its nucleic ...,Introduction,Introduction,Detection of SARS-CoV-2 RNA in commercial pass...,10.1093/jtm/taaa116
96247,10.1016/j.envres.2020.110265,bib4,Because the novel COVID-19 pandemic causes sev...,Because the novel COVID-19 pandemic causes sev...,Transmission route of SARS-CoV-2 into water an...,Transmission route of SARS-CoV-2 into water an...,Wearing masks and the fight against the novel ...,10.1016/j.pulmoe.2020.05.011


# Check if paper is retracted using CrossRef API:
 ### - Using type response in the json CrossRef sends back to see the if the status of the paper

In [10]:
CROSSREF_URL = "https://api.crossref.org/v1/works/"
def get_crossref(doi):
    url = f"{CROSSREF_URL}{doi}"
    response = requests.get(url, timeout = 5)

    if response.status_code == 200:
        data = response.json().get("message", {})
        r = data.get("update-to", "Not retracted")
        #print(data)
        if r != "Not retracted":
            if r[0]['type'] == 'retraction':
                print("Retracted")
                #print(r)
        else:
            print('Not retracted')

In [11]:
get_crossref('10.1128/MCB.00813-07')

Not retracted


# Multiple DOIs fetch response from CrossRef

In [33]:
test = dois[200:400]

In [12]:
import requests
import concurrent.futures
import pandas as pd
from tqdm import tqdm  

CROSSREF_URL = "https://api.crossref.org/v1/works/"

def get_crossref_metadata(doi):
    """Fetch metadata for a given DOI and check if it's retracted."""
    url = f"{CROSSREF_URL}{doi}"
    try:
        response = requests.get(url, timeout = 25)
        if response.status_code == 200:
            data = response.json().get("message", {})
            r = data.get("update-to", "Not retracted")
            if r != "Not retracted":
                if r[0].get('type') == 'retraction':
                    return "Retracted"
                else:
                    return r[0].get('type')
            else:
                return "Not retracted"
        else:
            return f"Error: {response.status_code}"
    except requests.exceptions.RequestException as e:
        return f"Request Error: {e}"

def check_dois_in_parallel(dois, batch_size=10, max_workers=1):
    """Processes DOIs in parallel and saves results in a new DataFrame column."""
   # dois = df[doi_column].tolist()
    results = {}

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_doi = {executor.submit(get_crossref_metadata, doi): doi for doi in dois}
        
        with tqdm(total=len(dois), desc="Checking Retractions", unit="DOI") as pbar:
            for future in concurrent.futures.as_completed(future_to_doi):
                doi = future_to_doi[future]
                status = future.result()
                results[doi] = status
                pbar.update(1)  # Update progress bar after each DOI

    #df["Retraction_Status"] = results  # Add results as a new column
    return results


status = check_dois_in_parallel(test)
#print(status)


# Extract the Retracted citation context of the DOIs using the retraction watch file from crossRef

In [46]:
# Check if the 
!make update

Checking for updates...
Local file is already up to date.


In [45]:
df = pd.read_csv('retraction_watch.csv',  dtype=str)
display(df)
#print(df['Reason'][df['OriginalPaperDOI'] == '10.1016/j.biopha.2018.03.122'].to_string(index = False))
#print(df.columns)

,Record ID,Title,Subject,Institution,Journal,Publisher,Country,Author,URLS,ArticleType,RetractionDate,RetractionDOI,RetractionPubMedID,OriginalPaperDate,OriginalPaperDOI,OriginalPaperPubMedID,RetractionNature,Reason,Paywalled,Notes
0,62940,Computer-aided cluster formation in wireless s...,(B/T) Data Science;(B/T) Technology;(PHY) Engi...,"Department of Information Technology, Sona Col...",Journal of Intelligent & Fuzzy Systems,IOS Press (bought by Sage November 2023),India,K Thangaraj;M Sakthivel;K Balasamy;S Suganyadevi,https://retractionwatch.com/2024/08/23/exclusi...,Research Article;,8/23/2024 0:00,10.3233/JIFS-219433,0,11/4/2023 0:00,10.3233/jifs-223242,0,Retraction,+Fake Peer Review;+Investigation by Journal/Pu...,No,An earlier publication date of the original ar...
1,62939,Computer graphics assisted methods in cultural...,(B/T) Technology;(HUM) Arts - General;,The Catholic University of Korea (Performing A...,Journal of Intelligent & Fuzzy Systems,IOS Press (bought by Sage November 2023),South Korea,Yuhan Wei;Youngju Kim,https://retractionwatch.com/2024/08/23/exclusi...,Research Article;,8/23/2024 0:00,10.3233/JIFS-219433,0,4/30/2024 0:00,10.3233/jifs-236320,0,Retraction,+Fake Peer Review;+Investigation by Journal/Pu...,No,"On the retraction notice, the publication date..."
2,62930,Comprehensive note on characterization of neut...,(PHY) Mathematics;,"Department of Mathematics, University of Scien...",Journal of Intelligent & Fuzzy Systems,IOS Press (bought by Sage November 2023),Italy;Jordan;Pakistan;Saudi Arabia,Arif Mehmood;Samer Al Ghour;Farkhanda Afzal;Gi...,https://retractionwatch.com/2024/08/23/exclusi...,Research Article;,8/23/2024 0:00,10.3233/JIFS-219433,0,6/1/2022 0:00,10.3233/jifs-212547,0,Retraction,+Fake Peer Review;+Investigation by Journal/Pu...,No,An earlier publication date of the original pu...
3,62929,Comparison of prediction fuzzy modeling toward...,(B/T) Data Science;(HSC) Medicine - Oncology;(...,"Department of Technology and Management, Facul...",Journal of Intelligent & Fuzzy Systems,IOS Press (bought by Sage November 2023),Malaysia,Aliya Syaffa Zakaria;Muhammad Ammar Shafi;Mohd...,https://retractionwatch.com/2024/08/23/exclusi...,Research Article;,8/23/2024 0:00,10.3233/JIFS-219433,0,4/18/2024 0:00,10.3233/jifs-233714,0,Retraction,+Fake Peer Review;+Investigation by Journal/Pu...,No,An earlier publication date of the original ar...
4,62928,Cloud computing encrypted image retrieval stra...,(B/T) Computer Science;(B/T) Technology;,"Department of Computer Science & Engineering, ...",Journal of Intelligent & Fuzzy Systems,IOS Press (bought by Sage November 2023),India;United States,R Sundar;M Purushotham Reddy;Abhisek Sethy;K S...,https://retractionwatch.com/2024/08/23/exclusi...,Research Article;,8/23/2024 0:00,10.3233/JIFS-219433,0,3/5/2024 0:00,10.3233/jifs-237948,0,Retraction,+Fake Peer Review;+Investigation by Journal/Pu...,No,An earlier publication date of the original ar...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62437,5,Effect of Perindopril on Large Artery Stiffnes...,(BLS) Biochemistry;(HSC) Medicine - Cardiology...,"Alfred and Baker Medical Unit, Baker Heart Res...",JAMA: Journal of the American Medical Association,American Medical Association,Australia,Anna A Ahimastos;Anuradha Aggarwal;Kellie M D'...,http://retractionwatch.com/2015/11/23/jama-ret...,Clinical Study;Research Article;,12/22/2015 0:00,10.1001/jama.2015.16678,26594834,10/3/2007 0:00,10.1001/jama.298.13.1539,1791149,Retraction,+Falsification/Fabrication of Data;+Investigat...,No,NaN
62438,4,MtvR is a global small noncoding regulatory RN...,(BLS) Biology - Cellular;(BLS) Genetics;(BLS) ...,Institute for Biotechnology and Bioengineering...,Journal of Bacteriology,American Society for Microbiology,Portugal,Christian G Ramos;André M Grilo;Paulo J P da C...,http://retractionwatch.com/2014/11/03/post-doc...,Research Article;,11/1/2014 0:00,10.1128/JB.02299-14,25319527,5/31/2013 0:00,10.11

In [38]:
""" Check if the given dois are in the retraction_watch.csv and 
    extract the reasons + add them to citation context df"""

merged_df = my_df.merge(df[['OriginalPaperDOI','Reason']], left_on='DOI', right_on='OriginalPaperDOI', how='inner').drop(columns=['OriginalPaperDOI'])
display(merged_df)


,Citing DOI,Citation ID,CC paragraph,CC prev_next,IMRD,Section title,Cited title,DOI,Reason
0,10.3390/ijms21197287,B74-ijms-21-07287,mTOR is a serine–threonine specificity protein...,"As a central regulator of cell growth, mTOR pl...",4. miRNAs and PA Therapy,4. miRNAs and PA Therapy,MicroRNA-145 inhibits the activation of the mT...,10.2147/OTT.S118391,+Concerns/Issues About Data;+Duplication of/in...
1,10.3390/ijms21197287,B74-ijms-21-07287,mTOR is a serine–threonine specificity protein...,"As a central regulator of cell growth, mTOR pl...",4. miRNAs and PA Therapy,4. miRNAs and PA Therapy,MicroRNA-145 inhibits the activation of the mT...,10.2147/OTT.S118391,+Concerns/Issues About Data;+Duplication of/in...
2,10.3390/molecules24193513,B24-molecules-24-03513,Early studies have suggested that Snail plays ...,"Snail, as an inhibitor of the E-cadherin gene,...",3. Discussion,3. Discussion,Nimotuzuma restrains proliferation and induces...,10.1002/jcp.28693,+Duplication of Data;+Duplication of/in Image;...
3,10.3390/cancers12113118,B12-cancers-12-03118,"Data on microRNAs, the sample source, the regu...","Data on microRNAs, the sample source, the regu...",3. Results,3. Results,MicroRNA-520a-3p suppresses epithelial-mesench...,10.1002/jcp.27199,+Concerns/Issues About Data;+Concerns/Issues a...
4,10.1186/s12885-022-09996-1,CR31,"In addition, Mertens C et al. report that unph...","Consistent with this notion, the correlation b...",Discussion,Discussion,MicroRNA-520a-3p suppresses epithelial-mesench...,10.1002/jcp.27199,+Concerns/Issues About Data;+Concerns/Issues a...
...,...,...,...,...,...,...,...,...,...
3832,10.1007/s12094-023-03189-3,CR89,The mechanisms of ncRNAs promoting chemotherap...,NcRNAs Expression Target gene Medicine Refs Ln...,NcRNAs regulate chemotherapeutic resistance in BC,NcRNAs promotes chemotherapeutic sensitivity,Long non-coding RNA LINC00968 attenuates drug ...,10.1186/s13046-019-1100-8,+Original Data and/or Images not Provided and/...
3833,10.1007/s12094-023-03189-3,CR89,WNT2 is a major Wnt ligand that regulates plac...,WNT2 is a major Wnt ligand that regulates plac...,NcRNAs regulate chemotherapeutic resistance in BC,LINC00968,Long non-coding RNA LINC00968 attenuates drug ...,10.1186/s13046-019-1100-8,+Error in Image;+Updated to Retraction;
3834,10.1007/s12094-023-03189-3,CR89,WNT2 is a major Wnt ligand that regulates plac...,WNT2 is a major Wnt ligand that regulates plac...,NcRNAs regulate chemotherapeutic resistance in BC,LINC00968,Long non-coding RNA LINC00968 attenuates drug ...,10.1186/s13046-019-1100-8,+Original Data and/or Images not Provided and/...
3835,10.1007/s12094-023-03189-3,CR223,Erb-2/Her-2 is up-regulated in 20–30% of human...,4 NcRNAs regulate response to targeted therapy...,NcRNAs with targeted therapy resistance in BC,NcRNAs with targeted therapy resistance in BC,miR-182 regulates trastuzumab resistance by ta...,10.1038/s41417-018-0031-4,+Concerns/Issues About Data;+Euphemisms for Pl...


In [35]:
merged_df.to_csv('Retracted_cc.tsv', sep='\t', index=False)